In [2]:
import cv2
import os
import shutil

# Input UTKFace folder
input_dir = "UTKFace"

# Output folders
long_dir = "hair_dataset/long"
short_dir = "hair_dataset/short"
os.makedirs(long_dir, exist_ok=True)
os.makedirs(short_dir, exist_ok=True)

# Load Haar face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def auto_label_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if len(faces) == 0:
        return None  # Skip if face not detected

    (x, y, w, h) = faces[0]
    face_bottom = y + h
    image_bottom = img.shape[0]
    below_face = image_bottom - face_bottom

    # Heuristic: long hair if enough space below the face
    if below_face > h * 0.8:
        return "long"
    else:
        return "short"

# Go through all images
for filename in os.listdir(input_dir):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        img_path = os.path.join(input_dir, filename)
        label = auto_label_image(img_path)

        if label == "long":
            shutil.copy(img_path, os.path.join(long_dir, filename))
        elif label == "short":
            shutil.copy(img_path, os.path.join(short_dir, filename))


KeyboardInterrupt: 

In [3]:
from tensorflow.keras.models import load_model

model = load_model("hair_length_model.h5")


In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam

# Configuration
IMG_SIZE = (128, 128)   # 👈 Change this to any fixed size you want (e.g. 224,224)
BATCH_SIZE = 32
EPOCHS = 10
HAIR_DIR = "hair_dataset"  # must contain 'short/' and 'long/' subfolders

# Image Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    HAIR_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    HAIR_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

# Model Definition
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D(),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary output
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train
model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS)

# Save model
model.save("hair_length_model.h5")
print("✅ Hair length model saved as hair_length_model.h5")

Found 6533 images belonging to 2 classes.
Found 1633 images belonging to 2 classes.


C:\Users\jaimi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\jaimi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 102s 492ms/step - accuracy: 0.9645 - loss: 0.1723 - val_accuracy: 0.9645 - val_loss: 0.1584
Epoch 2/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 51s 249ms/step - accuracy: 0.9645 - loss: 0.1577 - val_accuracy: 0.9645 - val_loss: 0.1572
Epoch 3/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 57s 277ms/step - accuracy: 0.9645 - loss: 0.1509 - val_accuracy: 0.9645 - val_loss: 0.1417
Epoch 4/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 55s 268ms/step - accuracy: 0.9645 - loss: 0.1423 - val_accuracy: 0.9645 - val_loss: 0.1324
Epoch 5/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 50s 242ms/step - accuracy: 0.9645 - loss: 0.1321 - val_accuracy: 0.9645 - val_loss: 0.1284
Epoch 6/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 50s 243ms/step - accuracy: 0.9645 - loss: 0.1243 - val_accuracy: 0.9645 - val_loss: 0.1385
Epoch 7/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 51s 248ms/step - accuracy: 0.9645 - loss: 0.1175 - val_accuracy: 0.9645 - val_loss: 0.1263
Epoch 8/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 50s 246ms/step - accuracy: 0.9645 - loss: 

✅ Hair length model saved as hair_length_model.h5


In [4]:
def predict_hair(image_path):
    # Load and preprocess
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # (1,128,128,3)

    # Predict
    prediction = model.predict(img_array)[0][0]
    label = "Long Hair" if prediction > 0.5 else "Short Hair"
    print(f"Prediction: {label} ({prediction:.4f})")
    return label

In [5]:
# After training your model
model.save("hair_length_model.h5")


In [6]:
# Save without optimizer state (recommended if only for inference)
model.save("my_model.h5", include_optimizer=False)
